In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from IPython.display import HTML
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import json
import math
import random

In [ ]:

folder_path = './assets/undistorted_05/eastbound'


files = os.listdir(folder_path)

def getNumber(x):
    num_w_ext = x.split('_')[2]
    num = num_w_ext.split('.')[0]
    return num

sorted_files = sorted(files, key=lambda x: int(getNumber(x)))

# print(sorted_files)

# points = [(1044, 784), (1439, 800), (1687, 1131), (229, 1034)]



# matrix = [[-2.55742823e-01, -2.45586077e+00,  2.03742378e+03], [ 9.30304363e-02, -2.62394313e+00,  1.80508106e+03], [ 4.87713313e-05, -1.54852038e-03,  1.00000000e+00]]

# matrix = [[-1.27785172e-01, -2.26968534e+00 , 1.73418419e+03],
#  [ 1.71129072e-01 ,-4.44935587e+00  ,2.88934325e+03],
#  [ 6.32873786e-05 ,-1.58006717e-03 , 1.00000000e+00]]

# matrix = np.float32(matrix)
src_points = np.float32([[1200,750], [1400,750], [2120,1515], [-1035, 1515]])
dst_points = np.float32([
    [1250,770],
    [1450,770],
    [1450, 1520],
    [1250, 1520]
])

M = cv2.getPerspectiveTransform(src_points, dst_points)
M_inv = cv2.getPerspectiveTransform(dst_points, src_points)








In [ ]:
normal_images = []
transformed_images = []

for img_source in sorted_files:
    print(img_source)
    image = cv2.imread(f"{folder_path}/{img_source}")

    transformed_image = cv2.warpPerspective(image, M, (image.shape[1], image.shape[0]))

    normal_images.append(image)
    transformed_images.append(transformed_image)

In [ ]:
plt.clf()

fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(16, 4))
colors = plt.cm.tab10.colors[:10]

path = "./assets/annotated_05/eastbound/"

x_data = []
y_data = []


def update(frame):
    print(frame)
    x_limit = (-500, 6000)
    y_limit = (-2000, 6000)

    ax1.clear()
    ax2.clear()
    ax3.clear()

    ax1.imshow(normal_images[frame])
    ax2.imshow(transformed_images[frame])  # Display the current frame for video 2

    name = sorted_files[frame].replace('png', 'json')

    with open(path + name, 'r') as json_file:
    # Load the JSON data
        data = json.load(json_file)

    for index, point in enumerate(data["pred_keypoints"]):
        x = point[0][0]
        # y = point[0][1]

        y = data["pred_boxes"][index][3]



        # Define the point you want to transform
        point = np.array([x, y])  # Example point at (50, 50)
        point_original = point.reshape(-1, 1, 2)
        transformed_point = cv2.perspectiveTransform(point_original, M)


        # print(frame, transformed_point, transformed_point_a)

        ax1.scatter(x, y, color=colors[index], s=50)
        ax2.scatter(transformed_point[0][0][0], transformed_point[0][0][1], color=colors[index], s=50)
        ax3.scatter(transformed_point[0][0][0], -transformed_point[0][0][1], color=colors[index], s=50)
        ax4.scatter(transformed_point[0][0][0], -transformed_point[0][0][1] + (frame * 40), color='black', s=2)

        # df.append({'X': transformed_point[0][0][0], 'Y': -transformed_point[0][0][1]}, ignore_index=True)

        x_data.append(transformed_point[0][0][0])
        y_data.append(-transformed_point[0][0][1] + (frame * 40))



    ax3.set_xlim(x_limit)
    ax3.set_ylim(y_limit)
    

    # ax4.set_xlim(x_limit)
    # ax4.set_ylim(y_limit)

    ax2.axis('off')  # Hide axes
    ax1.axis('off')  # Hide axes
    # ax3.figure.set_size_inches(5, 5)
    return ax1, ax2


# Create animations for both subplots
ani = animation.FuncAnimation(fig, update, frames=len(normal_images), interval=150)
# ani = animation.FuncAnimation(fig, update, frames=50, interval=150)

HTML(ani.to_jshtml())



In [ ]:
df = pd.DataFrame({'X': x_data, 'Y': y_data})
df

In [ ]:
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

X = df[['X', 'Y']]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled = X

# Step 4: Perform DBSCAN clustering
# You need to specify the epsilon (eps) parameter and the minimum samples (min_samples)
# You might need to experiment with these parameters to get the desired clustering results
eps = 100  # Adjust this based on your data
min_samples = 5  # Adjust this based on your data
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
clusters = dbscan.fit_predict(X_scaled)

# Step 5: Analyze the clustering results
# Add cluster labels to your DataFrame
df['Cluster'] = clusters

df_no_outlier = df[df['Cluster'] != -1]
# df_no_outlier = df

# Plot the clusters (assuming you have matplotlib installed)
plt.scatter(df_no_outlier['X'], df_no_outlier['Y'], c=df_no_outlier['Cluster'], cmap='viridis', s=5)
plt.xlabel('X')
plt.ylabel('Y')
plt.show()


plt.scatter(df_no_outlier['X'], df_no_outlier['Y'], c=df_no_outlier['Cluster'], cmap='viridis', s=5)
plt.xlabel('X')
plt.ylabel('Y')
plt.gca().set_aspect(1)
margin = 1000
plt.xlim(min(df_no_outlier['X']) - margin, max(df_no_outlier['X']) + margin)
plt.ylim(min(df_no_outlier['Y']) - margin, max(df_no_outlier['Y']) + margin)
plt.show()

df

In [ ]:
result_mean = df_no_outlier.groupby('Cluster').mean()
result_median = df_no_outlier.groupby('Cluster').median()


plt.scatter(df['X'], df['Y'], c='black', s=1)
plt.scatter(result_median['X'], result_median['Y'], c='red', s=20)
plt.show()


plt.scatter(result_mean['X'], result_mean['Y'], c='blue', s=4)
plt.scatter(result_median['X'], result_median['Y'], c='red', s=4)
plt.show()


plt.scatter(result_mean['X'], result_mean['Y'], c='blue', s=4)
plt.scatter(result_median['X'], result_median['Y'], c='red', s=4)
plt.gca().set_aspect(1)
margin = 1000  # Example margin value
plt.xlim(min(result_median['X']) - margin, max(result_median['X']) + margin)
plt.ylim(min(result_median['Y']) - margin, max(result_median['Y']) + margin)
plt.show()


result_median

In [ ]:
plt.clf()

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(8, 4))
colors = plt.cm.tab10.colors[:10]

path = "./assets/annotated_05/eastbound/"

x_data = []
y_data = []


# Function to update the image for video 2
def update(frame):
    print(frame)
    x_limit = (-500, 6000)
    y_limit = (-2000, 6000)

    ax1.clear()
    ax2.clear()

    ax1.imshow(normal_images[frame])
    ax2.imshow(transformed_images[frame])  # Display the current frame for video 2

    name = sorted_files[frame].replace('png', 'json')

    with open(path + name, 'r') as json_file:
    # Load the JSON data
        data = json.load(json_file)

    for index, point in enumerate(data["pred_keypoints"]):
        try:
            x = point[0][0]
            y = point[0][1]

            point = np.array([x, y])  # Example point at (50, 50)
            point_original = point.reshape(-1, 1, 2)
            transformed_point = cv2.perspectiveTransform(point_original, M)

            transformed_x = transformed_point[0][0][0]
            transformed_y = -transformed_point[0][0][1] + (frame * 40)


            result = df_no_outlier[(df_no_outlier['X'] == transformed_x) & (df_no_outlier['Y'] == transformed_y)]
            if len(result) >= 1:
                first_result = result.iloc[0]

                cluster_id = first_result['Cluster']

                first_test_df = result_median.iloc[int(cluster_id)]


                x_median = first_test_df['X']
                y_median = -(first_test_df['Y'] - (frame * 40))



                point_ab = np.array([x_median, y_median])
                point_original_ab = point_ab.reshape(-1, 1, 2)
                de_transformed_point = cv2.perspectiveTransform(point_original_ab, M_inv)

                test = colors[int(cluster_id % 10)]

                ax1.scatter(x, y, color=test, s=10)
                ax1.scatter(de_transformed_point[0][0][0], de_transformed_point[0][0][1], color="red", s=50)
                ax2.scatter(x_median, y_median, color="red", s=50)
                ax2.scatter(transformed_point[0][0][0], transformed_point[0][0][1], color=test, s=10)
            # print(test)

            # colors_cmap = [(0, 0, 0, 0), test]
            # cmap = LinearSegmentedColormap.from_list('Custom', colors_cmap, len(colors_cmap))

            # mask = masks[index]

            # ax1.imshow(mask, cmap=cmap, interpolation='nearest', alpha=1)
        except Exception as e:
            print("An unexpected error occurred:", e)


    ax2.axis('off')  # Hide axes
    ax1.axis('off')  # Hide axes
    # ax3.figure.set_size_inches(5, 5)
    return ax1, ax2


# Create animations for both subplots
ani = animation.FuncAnimation(fig, update, frames=len(normal_images), interval=150)

HTML(ani.to_jshtml())

In [ ]:
plt.clf()

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(8, 4))

def update(frame):
    ax1.clear()
    ax2.clear()

    ax2.imshow(normal_images[frame])

    shape = np.array(normal_images[frame]).shape

    height = shape[0]
    width = shape[1]

    cam_pos = np.array([width/2, height])

    cam_pos_reshaped = cam_pos.reshape(-1, 1, 2)

    transformed_cam_pos = cv2.perspectiveTransform(cam_pos_reshaped, M)

    ax1.scatter(result_median['X'], result_median['Y'], c='red', s=4)
    ax1.scatter(transformed_cam_pos[0][0][0], - transformed_cam_pos[0][0][1] + (frame*40), color="blue", s=50)
    
    ax2.axis('off')
    return ax1, ax2

ani = animation.FuncAnimation(fig, update, frames=len(normal_images), interval=150)

HTML(ani.to_jshtml())



In [ ]:
plt.clf()

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(8, 4))
colors = plt.cm.tab10.colors[:10]

path = "./assets/annotated_05/eastbound/"

cluster_list = []
widths  = []

# Function to update the image for video 2
def update(frame):
    print(frame)
    x_limit = (-500, 6000)
    y_limit = (-2000, 6000)

    ax1.clear()
    ax2.clear()

    ax1.imshow(normal_images[frame])
    ax2.imshow(transformed_images[frame])  # Display the current frame for video 2

    name = sorted_files[frame].replace('png', 'json')

    with open(path + name, 'r') as json_file:
    # Load the JSON data
        data = json.load(json_file)

    for index, point in enumerate(data["pred_keypoints"]):
        try:
            x = point[0][0]
            y = point[0][1]

            left_x = point[1][0]
            left_y = point[1][1]

            right_x = point[2][0]
            right_y = point[2][1]


            left_point = np.array([left_x, left_y])
            left_point_o = left_point.reshape(-1, 1, 2)
            transformed_left_point = cv2.perspectiveTransform(left_point_o, M)

            right_point = np.array([right_x, right_y])
            right_point_o = right_point.reshape(-1, 1, 2)
            transformed_right_point = cv2.perspectiveTransform(right_point_o, M)

            point = np.array([x, y])
            point_original = point.reshape(-1, 1, 2)
            transformed_point = cv2.perspectiveTransform(point_original, M)

            transformed_x = transformed_point[0][0][0]
            transformed_y = -transformed_point[0][0][1] + (frame * 40)

            ax1.scatter(x, y, color="white", s=5)
            ax1.scatter(left_x, left_y, color="blue", s=5)
            ax1.scatter(right_x, right_y, color="red", s=5)

            ax2.scatter(transformed_left_point[0][0][0], transformed_left_point[0][0][1], color="blue", s=5)
            ax2.scatter(transformed_right_point[0][0][0], transformed_right_point[0][0][1], color="red", s=5)

            euclidean_distance = math.sqrt((transformed_left_point[0][0][0] - transformed_right_point[0][0][0])**2 + (transformed_left_point[0][0][1] - transformed_right_point[0][0][1])**2)

            print(euclidean_distance)

            result = df_no_outlier[(df_no_outlier['X'] == transformed_x) & (df_no_outlier['Y'] == transformed_y)]
            if len(result) >= 1:
                first_result = result.iloc[0]

                cluster_id = first_result['Cluster']

                cluster_list.append(int(cluster_id))
                widths.append(euclidean_distance)

        except Exception as e:
            print("An unexpected error occurred:", e)

    

    ax2.axis('off')  # Hide axes
    ax1.axis('off')  # Hide axes
    return ax1, ax2



ani = animation.FuncAnimation(fig, update, frames=len(normal_images), interval=150)

# ani = animation.FuncAnimation(fig, update, frames=5, interval=150)

HTML(ani.to_jshtml())

In [ ]:
df_width_trees = pd.DataFrame({'Cluster': cluster_list, 'Width_px': widths})

df_width_trees_median = df_width_trees.groupby('Cluster').median()

df_width_trees_in_cm = df_width_trees_median.assign(Width_cm=df_width_trees_median['Width_px'] * 2)

df_trees = result_median.merge(df_width_trees_in_cm, left_on='Cluster', right_on='Cluster')

df_trees

In [ ]:
plt.scatter(df_trees['X'], df_trees['Y'], c='brown', s=df_trees['Width_px'])
plt.gca().set_aspect(1)

margin = 1000
plt.xlim(min(df_trees['X']) - margin, max(df_trees['X']) + margin)
plt.ylim(min(df_trees['Y']) - margin, max(df_trees['Y']) + margin)

plt.show()

In [ ]:
im_1 = "assets/undistorted_05/eastbound/eastbound_20240319_07816.png"
im_2 = "assets/undistorted_05/eastbound/eastbound_20240319_07821.png"


img1 = cv2.imread(im_1, cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread(im_2, cv2.IMREAD_GRAYSCALE)

# Create SIFT detector
sift = cv2.SIFT_create()

# Detect keypoints and compute descriptors
keypoints1, descriptors1 = sift.detectAndCompute(img1, None)
keypoints2, descriptors2 = sift.detectAndCompute(img2, None)

# Define FLANN parameters for descriptor matching
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)

# Create FLANN matcher
flann = cv2.FlannBasedMatcher(index_params, search_params)

# Match descriptors
matches = flann.knnMatch(descriptors1, descriptors2, k=2)

# Lowe's ratio test
good_matches = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        good_matches.append(m)

# Limit matches for better visualization
if len(good_matches) > 100:
    limited_matches = random.sample(good_matches, 100)

# Draw matches
img_matches_full = cv2.drawMatches(img1, keypoints1, img2, keypoints2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

img_matches_limited = cv2.drawMatches(img1, keypoints1, img2, keypoints2, limited_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)



plt.figure(figsize=(18, 9))
plt.imshow(cv2.cvtColor(img_matches_limited, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

plt.figure(figsize=(18, 9))
plt.imshow(cv2.cvtColor(img_matches_full, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

In [ ]:
# im_1 = "assets/undistorted_05/eastbound/eastbound_20240319_07816.png"
# im_2 = "assets/undistorted_05/eastbound/eastbound_20240319_07821.png"

img1 = normal_images[0]
img2 = normal_images[1]


match = good_matches[100]

query_idx = match.queryIdx  # Index of the matched keypoint in the query set
train_idx = match.trainIdx  # Index of the matched keypoint in the train set

# Get the coordinates of the matched keypoints using their indices
query_keypoint = keypoints1[query_idx]  # Assuming 'query_keypoints' is a list of keypoints in the query image
train_keypoint = keypoints2[train_idx]  # Assuming 'train_keypoints' is a list of keypoints in the train image

# Extract the (x, y) coordinates of the matched keypoints
query_x, query_y = query_keypoint.pt
train_x, train_y = train_keypoint.pt


plt.figure(figsize=(6, 4))
plt.imshow(img1)
plt.scatter(query_x, query_y, color="red")
plt.axis('off')
plt.show()

plt.figure(figsize=(6, 4))
plt.imshow(img2)
plt.scatter(train_x, train_y, color="red")
plt.axis('off')
plt.show()


In [ ]:
point_a = np.array([query_x, query_y])
point_original_a = point_a.reshape(-1, 1, 2)
transformed_point_a = cv2.perspectiveTransform(point_original_a, M)


point_b = np.array([train_x, train_y])
point_original_b = point_b.reshape(-1, 1, 2)
transformed_point_b = cv2.perspectiveTransform(point_original_b, M)

img1_trans = transformed_images[0]
img2_trans = transformed_images[1]


plt.figure(figsize=(6, 4))
plt.imshow(img1_trans)
plt.scatter(transformed_point_a[0][0][0], transformed_point_a[0][0][1], color="red")
plt.axis('off')
plt.show()

plt.figure(figsize=(6, 4))
plt.imshow(img2_trans)
plt.scatter(transformed_point_b[0][0][0], transformed_point_b[0][0][1], color="red")
# plt.axis('off')
plt.show()

In [ ]:
print(transformed_point_a)
print(transformed_point_b)

In [ ]:
x_diff = transformed_point_b[0][0][0] - transformed_point_a[0][0][0]

y_diff = transformed_point_b[0][0][1] - transformed_point_a[0][0][1]

print(x_diff, y_diff)

In [ ]:
x_diff_list = []
y_diff_list = []

for match in good_matches:
    query_idx = match.queryIdx  # Index of the matched keypoint in the query set
    train_idx = match.trainIdx  # Index of the matched keypoint in the train set

    # Get the coordinates of the matched keypoints using their indices
    query_keypoint = keypoints1[query_idx]  # Assuming 'query_keypoints' is a list of keypoints in the query image
    train_keypoint = keypoints2[train_idx]  # Assuming 'train_keypoints' is a list of keypoints in the train image

    # Extract the (x, y) coordinates of the matched keypoints
    query_x, query_y = query_keypoint.pt
    train_x, train_y = train_keypoint.pt

    point_a = np.array([query_x, query_y])
    point_original_a = point_a.reshape(-1, 1, 2)
    transformed_point_a = cv2.perspectiveTransform(point_original_a, M)


    point_b = np.array([train_x, train_y])
    point_original_b = point_b.reshape(-1, 1, 2)
    transformed_point_b = cv2.perspectiveTransform(point_original_b, M)

    if((transformed_point_b[0][0][1] < 1520 or transformed_point_a[0][0][1] < 1520) and (transformed_point_b[0][0][1] > 0 or transformed_point_a[0][0][1] > 0) and (transformed_point_b[0][0][0] < 2560 or transformed_point_a[0][0][0] < 2560) and (transformed_point_b[0][0][0] > 0 or transformed_point_a[0][0][0] > 0) ):
        img1_trans = transformed_images[0]
        img2_trans = transformed_images[1]

        x_diff = transformed_point_b[0][0][0] - transformed_point_a[0][0][0]

        y_diff = transformed_point_b[0][0][1] - transformed_point_a[0][0][1]

        print(x_diff, y_diff)


        x_diff_list.append(x_diff)
        y_diff_list.append(y_diff)

In [ ]:
import statistics

mean_x = statistics.mean(x_diff_list)
mean_y = statistics.mean(y_diff_list)

print(mean_x, mean_y)

median_x = statistics.median(x_diff_list)
median_y = statistics.median(y_diff_list)

print(median_x, median_y)